In [1]:
import os
import numpy as np
import pandas as pd
from datetime import timedelta, datetime
from tqdm import tqdm

In [2]:
FORBES = './stock_market_data/sp500/csv/'
TICKER = 'A'
PATH = os.path.join(FORBES, f"{TICKER}.csv")

In [50]:
df = pd.read_csv(PATH)
df['Date'] = pd.to_datetime(df['Date'])

In [6]:
df = df.drop(['Date'], axis=1)

In [7]:
df.describe()

,Low,Open,Volume,High,Close,Adjusted Close
count,5693.000000,5693.000000,5.693000e+03,5693.000000,5693.000000,5693.000000
mean,42.660654,43.217349,3.495960e+06,43.774675,43.226476,40.779416
std,33.196632,33.561343,2.439198e+06,33.895923,33.542048,33.884099
min,7.510730,7.653791,2.221500e+05,7.961373,7.761087,6.652129
25%,21.673820,21.959944,1.992800e+06,22.281832,22.002861,19.392338
50%,29.713877,30.164520,2.983100e+06,30.579399,30.171675,27.384373
75%,52.259998,52.790001,4.319400e+06,53.419998,52.709999,48.434860
max,177.169998,179.279999,6.254638e+07,179.570007,179.279999,178.225525


In [19]:
def make_dataset(df: np.ndarray, time_d: int = 10):
    X, y = [], []

    for i in range(df.shape[0] - time_d - 1):
        idx = i + time_d
        row = df[idx + 1]
        row = row[:-1] # remove volume is it doesn't need to be predicted
        x = df[i:idx]
        if any(np.isnan(row)) or np.isnan(row).any():
            continue # if the row (y) has nans, skip

        X.append(x.values)
        y.append(row.values)
        
    return np.array(X), np.array(y)

In [20]:
X, y = make_dataset(df.to_numpy())

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [31]:
X.shape

(5682, 10, 6)

In [4]:
def get_all_csv_files(dir: str):
    files = []
    a = os.listdir(dir)
    for b in a:
        if os.path.isdir(os.path.join(dir, b)):
            c = os.path.join(dir, b, 'csv')
            if os.path.isdir(c):
                files += [os.path.join(c, f) for f in os.listdir(c) if f[-4:] == '.csv']

    return files

In [5]:
def large_df(files):
    return pd.concat((pd.read_csv(f) for i, f in tqdm(enumerate(files), total=len(files))), ignore_index=True)

In [6]:
def scale(X, y):
    div = 255 / np.max(X)
    X *= div
    y *= div
    return X, y

In [7]:
df = large_df(get_all_csv_files('stock_market_data'))

100%|██████████| 4281/4281 [00:34<00:00, 124.38it/s]


In [8]:
df.shape

(24603272, 7)

In [9]:
from sklearn.preprocessing import MinMaxScaler
import pickle

scl = MinMaxScaler()
scl.fit(df.drop(['Date'], axis=1))

MinMaxScaler()

In [10]:
pickle.dump(scl, open('minmax_scaler.bin', 'wb'))

In [52]:
x = scl.transform(df.drop(['Date'], axis=1))

In [58]:
ddf = pd.DataFrame(x, columns=['Low', 'Open', 'Volume', 'High', 'Close', 'Adjusted Close'])

In [59]:
ddf.describe()

,Low,Open,Volume,High,Close,Adjusted Close
count,5.693000e+03,5.693000e+03,5693.000000,5.693000e+03,5.693000e+03,5693.000000
mean,5.124624e-12,5.191498e-12,0.000471,5.258447e-12,5.192594e-12,0.998474
std,3.987756e-12,4.031567e-12,0.000329,4.071758e-12,4.029249e-12,0.000000
min,9.022288e-13,9.194140e-13,0.000030,9.563624e-13,9.323030e-13,0.998474
25%,2.603574e-12,2.637945e-12,0.000269,2.676612e-12,2.643101e-12,0.998474
50%,3.569389e-12,3.623522e-12,0.000402,3.673360e-12,3.624382e-12,0.998474
75%,6.277749e-12,6.341415e-12,0.000582,6.417094e-12,6.331805e-12,0.998474
max,2.128260e-11,2.153607e-11,0.008428,2.157090e-11,2.153607e-11,0.998474


In [61]:
ddf[['Low', 'Open', 'High', 'Close', 'Adjusted Close', 'Volume']].describe()

,Low,Open,High,Close,Adjusted Close,Volume
count,5.693000e+03,5.693000e+03,5.693000e+03,5.693000e+03,5693.000000,5693.000000
mean,5.124624e-12,5.191498e-12,5.258447e-12,5.192594e-12,0.998474,0.000471
std,3.987756e-12,4.031567e-12,4.071758e-12,4.029249e-12,0.000000,0.000329
min,9.022288e-13,9.194140e-13,9.563624e-13,9.323030e-13,0.998474,0.000030
25%,2.603574e-12,2.637945e-12,2.676612e-12,2.643101e-12,0.998474,0.000269
50%,3.569389e-12,3.623522e-12,3.673360e-12,3.624382e-12,0.998474,0.000402
75%,6.277749e-12,6.341415e-12,6.417094e-12,6.331805e-12,0.998474,0.000582
max,2.128260e-11,2.153607e-11,2.157090e-11,2.153607e-11,0.998474,0.008428


In [18]:
df.iloc[:10].values[:, :5]

array([['18-11-1999', 28.612302780151367, 32.54649353027344, 62546380.0,
        35.765380859375],
       ['19-11-1999', 28.47818374633789, 30.713518142700195, 15234146.0,
        30.75822639465332],
       ['22-11-1999', 28.65700912475586, 29.551143646240234, 6577870.0,
        31.473533630371094],
       ['23-11-1999', 28.612302780151367, 30.40057182312012, 5975611.0,
        31.205293655395508],
       ['24-11-1999', 28.612302780151367, 28.701717376708984, 4843231.0,
        29.99821281433105],
       ['26-11-1999', 29.14878463745117, 29.23819732666016, 1729466.0,
        29.685264587402344],
       ['29-11-1999', 29.014663696289062, 29.327611923217773, 4074751.0,
        30.355865478515625],
       ['30-11-1999', 29.28290367126465, 30.042917251586918, 4310034.0,
        30.713518142700195],
       ['01-12-1999', 29.95350456237793, 30.177038192749023, 2957329.0,
        31.0711727142334],
       ['02-12-1999', 30.8923454284668, 31.29470634460449, 3069868.0,
        32.1888427734375]